<a href="https://colab.research.google.com/github/bilpang28/AMAZON_AI_AND_ML/blob/main/STKI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Inisialisasi Program**

Pada program ini memvisualisasikan hasil prediksi pada peta atau melibatkan data spasial dari data local yang diinputkan dalam code. Sebelum melakukan pencodingan maka lakukan installasi environment seperti berikut ini:

In [298]:
!pip install pandas numpy

In [299]:
import pandas as pd
import numpy as np

# Input Data dan Ekstraksi Data

Setelah import semua library yang digunakan kemudian masukkan data secara lokal dengan daftar sebagai berikut :
1. daerah berisi nama daerah dari tarakan, bulungan, tanatidung
2. TPS berisi data jumlah tps di tarakan, bulungan, tanatidung
3. DPT berisi data jumlah dpt di setiap daerah di kalimantan utara
4. cadangan dpt 2 % ini adalah data yang digunakan untuk memastikan bahwa jika kekurangan dpt akan ditambahkan kedalam DPT yang kekurangan
5. suara, dataset ini adalah jumlah suara yang terkumpul dari hasil pengumpulan pencoblosan disetiap daerah di kalimantan utara
6. latitude digunakan untuk menentukan garis lintang dari daerah kalimantan utara
7. longitude adalah yang digunakan untuk menarik garis bujur daerah kalimantan utara

In [300]:
data = {
    'Daerah': ['Tarakan', 'Bulungan', 'Tanatidung'],
    'Tps': [628, 436, 73],
    'Dpt': [154264, 95633, 14968],
    'cadangan_Dpt': [3335, 2112, 330],
    'Jumlah_Suara': [157599, 97745, 15298],
    'Latitude': np.random.uniform(-0.5, 0.5, 3) + 0.5,
    'Longitude': np.random.uniform(117591, 116985, 3),
    'Target_Value': np.random.rand(3),
}

In [301]:
df = pd.DataFrame(data)
print(df)

       Daerah  Tps     Dpt  cadangan_Dpt  Jumlah_Suara  Latitude  \
0     Tarakan  628  154264          3335        157599  0.374540   
1    Bulungan  436   95633          2112         97745  0.950714   
2  Tanatidung   73   14968           330         15298  0.731994   

       Longitude  Target_Value  
0  117228.212959      0.058084  
1  117496.452704      0.866176  
2  117496.467321      0.601115  


# Compare GIR dan Data

Inputan data dengan Latitude dan Longitude untuk menghitung hasil GIR dengan rumus Euclidean sebagai cara mencari jarak dari lokasi Tarakan untuk mencari K nya.

In [302]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

for index_i, row_i in df.iterrows():
    for index_j, row_j in df.loc[index_i+1:].iterrows():
        distance = euclidean_distance(row_i['Latitude'], row_i['Longitude'],
                                      row_j['Latitude'], row_j['Longitude'])
        print(f"Euclidean distance between {row_i['Daerah']} and {row_j['Daerah']}: {distance}")

Euclidean distance between Tarakan and Bulungan: 268.24036412051953
Euclidean distance between Tarakan and Tanatidung: 268.25460025643184
Euclidean distance between Bulungan and Tanatidung: 0.2192082304012567


# Partisi Data

1. Train Test Data

    Membagi data menjadi data training dan data testing. Data dibagi menjadi 80 : 20 dengan komposisi 80 persen Data Training dengan data testing 20 persen. random state untuk reproducibility 42.

In [303]:
np.random.seed(80)

Melakukan random pemisahan data

In [304]:
indices = np.random.permutation(df.index)

Mengatur jumlah data training sebanyak 80 persen dari data keseluruhan.

In [305]:
train_size = int(0.8 * len(df))

Memasukkan data kedalam proses training dan testing data

In [306]:
train_indices, test_indices = indices[:train_size], indices[train_size:]

Membuat Dataframes untuk training dan testing data

In [307]:
train_df, test_df = df.loc[train_indices], df.loc[test_indices]

2. K-Fold Cross Validation

    K-fold adalah salah satu metode Cross Validation yang populer dengan melipat data sebanyak K dan mengulangi (men-iterasi) experimennya sebanyak K juga.

In [325]:
k_folds = 3
fold_indices = np.array_split(indices, k_folds)

In [326]:
for fold in range(k_folds):

    test_indices = fold_indices[fold]
    train_indices = np.concatenate([fold_indices[i] for i in range(k_folds) if i != fold])

    train_df, test_df = df.loc[train_indices], df.loc[test_indices]

Menampilkan hasil training dan testing data

In [327]:
    print(f"\nFold {fold + 1}:")
    print("Training Set:")
    print(train_df)
    print("\nTesting Set:")
    print(test_df)


Fold 3:
Training Set:
       Daerah  Tps    Dpt  cadangan_Dpt  Jumlah_Suara  Latitude  Longitude  \
1    Bulungan  436  95633          2112         97745  1.000000   0.999946   
2  Tanatidung   73  14968           330         15298  0.620392   1.000000   

   Target_Value  
1      0.866176  
2      0.601115  

Testing Set:
    Daerah  Tps     Dpt  cadangan_Dpt  Jumlah_Suara  Latitude  Longitude  \
0  Tarakan  628  154264          3335        157599       0.0        0.0   

   Target_Value  
0      0.058084  


# Normalisasi Data
Memproses Data yang mengubah nilai-nilai dalam dataset menjadi skala yang seragam atau dalam rentang tertentu untuk mencapai metode KNN. Menggunakan Min-Max Scalling.

In [311]:
def min_max_scaling(feature):
    min_val = np.min(feature)
    max_val = np.max(feature)
    scaled_feature = (feature - min_val) / (max_val - min_val)
    return scaled_feature

In [312]:
features_to_normalize = ['Latitude', 'Longitude']

In [313]:
for feature in features_to_normalize:
    df[feature] = min_max_scaling(df[feature])

In [314]:
print("Normalized Data:")
print(df)

Normalized Data:
       Daerah  Tps     Dpt  cadangan_Dpt  Jumlah_Suara  Latitude  Longitude  \
0     Tarakan  628  154264          3335        157599  0.000000   0.000000   
1    Bulungan  436   95633          2112         97745  1.000000   0.999946   
2  Tanatidung   73   14968           330         15298  0.620392   1.000000   

   Target_Value  
0      0.058084  
1      0.866176  
2      0.601115  


# Model KNN

Tipe algoritma pembelajaran berbasis instan, yang berarti modelnya tidak dilatih secara eksplisit. Sebaliknya, saat melakukan prediksi, algoritma ini mencari k-neighbors terdekat dalam ruang fitur dan mengambil mayoritas label atau nilai dari tetangga tersebut untuk memprediksi label atau nilai yang sesuai untuk instan baru.

In [315]:
def knn(train_data, test_point, k):
    distances = []

    for index, row in train_data.iterrows():
        train_point = np.array([row['Latitude'], row['Longitude']])
        dist = euclidean_distance(test_point[0], test_point[1], train_point[0], train_point[1])
        distances.append((index, dist))

    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]

    class_counts = {}
    for neighbor in neighbors:
        label = train_data.loc[neighbor[0], 'Target_Value']
        if label in class_counts:
            class_counts[label] += 1
        else:
            class_counts[label] = 1

    majority_label = max(class_counts, key=class_counts.get)
    return majority_label

In [316]:
def train_knn(train_data, k):
    def predict(test_point):
        return knn(train_data, test_point, k)

    return predict

In [329]:
k_value = 3
test_point = np.array([0.3, 0.9])
knn_model = train_knn(train_df, k_value)
predicted_label = knn_model(test_point)
print(f"Predicted Label for the test point: {predicted_label}")

Predicted Label for the test point: 0.6011150117432088


Accuracy: 0.0
Precision: 0
Recall: 0
F1 Score: 0
